<a href="https://colab.research.google.com/github/YahyaMansoor/Cryptography/blob/main/Assignment_3_Cryptography_Cr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from typing import List, Tuple
from Crypto.Cipher import AES

def convert_hex_str_to_int_list(a: str) -> List[int]:
    return [int(a[i: i+2], 16) for i in range(0, len(a), 2)]

def extract_iv_and_cipher(cipher_list: List[int], iv_length: int) -> Tuple[List[int], List[int]]:

    return (cipher_list[:iv_length], cipher_list[iv_length:])

def compute_block_num(key: List[int], cipher: List[int]) -> int:

    return round(len(cipher) / len(key))

def cbc_decryption(key: str, cipher: str) -> str:

    key_list = convert_hex_str_to_int_list(key)
    block_size = len(key_list)
    cipher_list = convert_hex_str_to_int_list(cipher)
    iv_list, cipher_list = extract_iv_and_cipher(cipher_list, block_size)
    block_num = compute_block_num(key_list, cipher_list)
    aes_decoder = AES.new(bytes(key_list), AES.MODE_ECB)
    iv = bytes(iv_list)
    result_list = []
    for i in range(block_num):
        result_need_xor = aes_decoder.decrypt(bytes(cipher_list[i*block_size: (i+1) * block_size]))
        result = [iv[j] ^ result_need_xor[j] for j in range(len(result_need_xor))]
        result_list.extend(result)
        iv = cipher_list[i*block_size: (i+1) * block_size]
    return "".join([chr(x) for x in result_list[: len(result_list) - result_list[len(result_list)-1]]])

def ctr_iv_add(key: List[int], step: int) -> List[int]:

    key_len = len(key)
    for i in range(key_len - 1, -1, -1):
        j = key[i] + step
        if j % 0xFF == j:
            key[i] = j
            break
        else:
            key[i] = j % 0xFF
            step = int(j / 0xFF)
    return key

def ctr_decryption(key: str, cipher: str) -> str:

    key_list = convert_hex_str_to_int_list(key)
    block_size = len(key_list)
    cipher_list = convert_hex_str_to_int_list(cipher)
    iv_list, cipher_list = extract_iv_and_cipher(cipher_list, block_size)
    block_num = compute_block_num(key_list, cipher_list)
    result_list = []
    aes_encoder = AES.new(bytes(key_list), AES.MODE_ECB)
    for i in range(block_num):
        round_iv = ctr_iv_add(iv_list[:], i)
        iv_cipher = aes_encoder.encrypt(bytes(round_iv))
        cipher_block = cipher_list[i*block_size: (i+1) * block_size]
        result = [iv_cipher[j] ^ cipher_block[j] for j in range(len(cipher_block))]
        result_list.extend(result)
    return "".join([chr(x) for x in result_list])

if __name__ == "__main__":

    print("CBC mode:")

    key = "140b41b22a29beb4061bda66b6747e14"
    ciphertext = "4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81"
    result = cbc_decryption(key, ciphertext)
    print("Question 1: {}".format(result))

    key = "140b41b22a29beb4061bda66b6747e14"
    ciphertext = "5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253"
    result = cbc_decryption(key, ciphertext)
    print("Question 2: {}".format(result))

    print("CTR mode:")

    key = "36f18357be4dbd77f050515c73fcf9f2"
    ciphertext = "69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329"
    result = ctr_decryption(key, ciphertext)
    print("Question 3: {}".format(result))

    key = "36f18357be4dbd77f050515c73fcf9f2"
    ciphertext = "770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451"
    result = ctr_decryption(key, ciphertext)
    print("Question 4: {}".format(result))

CBC mode:
Question 1: Basic CBC mode encryption needs padding.
Question 2: Our implementation uses rand. IV
CTR mode:
Question 3: CTR mode lets you build a stream cipher from a block cipher.
Question 4: Always avoid the two time pad!
